In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
data = pd.read_csv('DATA_PROVINSI.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.7 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon = individual
    if C <= 0 or epsilon <= 0 :
        return float('inf'),
    svr = SVR(kernel='linear', C=C, epsilon=epsilon)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
C_range = (10, 50)
epsilon_range = (0.001, 5)

def create_individual():
    C = random.uniform(*C_range)
    epsilon = random.uniform(*epsilon_range)
    return creator.Individual([C, epsilon])
    
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.5)  # Menggunakan crossover uniform 
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2) # Menggunakan mutasi gaussian
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 240
population = toolbox.population(n=population_size)

# Genetic algorithm parameters    
num_generations = 90
crossover_rate = 0.3
mutation_rate = 0.7

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    for ind in offspring:
        ind[0] = max(min(ind[0], C_range[1]), C_range[0])
        ind[1] = max(min(ind[1], epsilon_range[1]), epsilon_range[0])
        
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon = best_individual

# Test the best SVR model
best_svr = SVR(kernel='linear', C=C, epsilon=epsilon)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"MAPE: {mape:.4f}")

Generasi 1:
Individu: [45.20428929028596, 3.1674615142904115], MAPE: 24.428362554007556
Individu: [25.43151548609408, 0.5341714414474565], MAPE: 24.428362554007556
Individu: [24.70703140796482, 1.9765662868837732], MAPE: 24.428362554007556
Individu: [40.19238126054765, 0.4547309719012145], MAPE: 24.428362554007556
Individu: [24.522321239848953, 0.23632804235559163], MAPE: 29.134289111249757
Individu: [26.030231461082224, 4.750159940432964], MAPE: 24.428362554007556
Individu: [38.54806685916278, 3.2356457707860384], MAPE: 24.428362554007556
Individu: [39.556433917817905, 0.40112777607276434], MAPE: 25.117485162895463
Individu: [38.02149351139174, 2.7813958045313956], MAPE: 24.428362554007556
Individu: [41.507738725629, 0.06578883830637824], MAPE: 33.50760899171756
Individu: [41.74186150671496, 0.8220238719949299], MAPE: 24.428362554007556
Individu: [11.191561099028808, 2.8430580430479853], MAPE: 24.428362554007556
Individu: [43.553690342756916, 2.80109767031114], MAPE: 24.42836255400755

In [6]:
# Prediksi 6 bulan ke depan
last_value = df['jumlah_kecelakaan'].values[-1] # Nilai terakhir dari data sebenarnya
predicts = []

for i in range(6):
    next_value = best_svr.predict(np.array([[last_value]]))[0]
    predicts.append(next_value)
    last_value = next_value

# Denormalisasi prediksi
predicts = scaler.inverse_transform(np.array(predicts).reshape(-1, 1))

print("Prediksi untuk 6 bulan ke depan:")
print(predicts)

Prediksi untuk 6 bulan ke depan:
[[18.25955963]
 [18.20464865]
 [18.20057646]
 [18.20027447]
 [18.20025207]
 [18.20025041]]
